In [1]:
import torch
from torch import nn
from torch.nn import init
import torch.utils.data as data_utils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import heapq

In [2]:
#dataset = np.loadtxt('../Yelp/yelp.rating', usecols=[0,1,3], dtype=int)
dataset = np.loadtxt('../ml-1m/ratings.dat', delimiter='::', usecols=[0,1,3], dtype=int)

In [3]:
FM_SIZE = [64,32,16,8,4,2,1]
N_FM = 32
N_USERS = np.max(dataset[:,0])
N_ITEMS = np.max(dataset[:,1])
LEARNING_RATE = 0.001
BATCH_SIZE = 512
EPOCH = 60

In [4]:
users_items = np.zeros((N_USERS+1,N_ITEMS+1), dtype=np.int8)
for line in dataset:
    users_items[line[0],line[1]] = 1
users_items_train = users_items.copy()

In [5]:
u_pos_test = list() # 作为测试集的交互正例
for i in range(N_USERS+1):
    if i==0: 
        continue
    uitems = dataset[dataset[:,0]==i]
    onepos = uitems[uitems[:,-1]==np.max(uitems),:2][0]   # 每个用户取时间戳最大的交互
    u_pos_test.append(onepos)
    users_items_train[onepos[0], onepos[1]] = 0

In [32]:
def generate_train_batch(interact_matrix, test, shuffle = True, drop_last = False, batch_size = BATCH_SIZE):
    """
    构造训练用的三元组
    对于随机抽出的用户u，i可以从user_ratings随机抽出，而j也是从总的电影集中随机抽出，当然j必须保证(u,j)不在user_ratings中
    """
    n_users = interact_matrix.shape[0]
    n_items = interact_matrix.shape[1]
    one_epoch = list()   # 当前epoch
    index = np.array(range(n_users))
    if shuffle:
        np.random.shuffle(index)  # 用户索引shuffle
    for start_index in range(0, n_users, batch_size):
        if (start_index+batch_size) > n_users and drop_last:  # 不保留最后一个不满batch_size长度的batch
            continue
        end_index = n_users if (start_index+batch_size) > n_users else (start_index+batch_size)
        users = index[start_index: end_index]   # 当前batch的用户编号
        temp = interact_matrix[users]           # 当前batch用户的所有正负交互
        one_batch = list()  # 当前batch
        for u, line in enumerate(temp):
            if users[u] == 0:
                continue
            P = np.nonzero(line)[0]                # 该用户users[u]的所有正例
            N = list(set(range(n_items))^set(P))   # 该用户users[u]的所有负例
            i = np.random.choice(P, 1)[0]
            j = np.random.choice(N, 1)[0]
            '''
            while j == test[users[u]][1]:    # 若j存在于测试集正例中
                j = np.random.choice(N, 1)[0]
            '''
            one_batch.append([users[u],i,j])
        one_epoch.append(np.array(one_batch))
    return np.array(one_epoch)

In [33]:
def generate_test_batch(users_items, u_pos_test, negatives_sample_size = 999):
    utest = list()
    itest = list()
    for ui in u_pos_test:
        u = ui[0]
        i = ui[1]    
        P = np.nonzero(users_items[u])[0]
        N = list(set(range(N_ITEMS+1))^set(P))
        negatives_sample = np.random.choice(N, negatives_sample_size)  # 负采样 -- 不放回
        negatives = [i]  # 正例
        for n in negatives_sample:
            negatives.append(n)  # 添加负例
        utest.append([u for j in range(negatives_sample_size+1)])
        itest.append(negatives)
    return np.array(utest), np.array(itest)
utest, itest = generate_test_batch(users_items, u_pos_test)

In [34]:
class ConvNCF(nn.Module):
    def __init__(self, fm_sizes, n_users, n_items, n_fm=N_FM, myStride=2, n_output=1):
        ''' e.g.--> fm_sizes = [64,32,16,8,4,2,1] '''
        super(ConvNCF, self).__init__()
        self.convs = list()
        self.dropout = nn.Dropout(p=0.5)
        self.user_embedding_layer = nn.Embedding(n_users+1, fm_sizes[0])
        self._set_normalInit(self.user_embedding_layer, hasBias = False) 
        #self._set_xavierInit(self.user_embedding_layer, hasBias = False)
        #self._set_heInit(self.user_embedding_layer, hasBias = False) 
        self.item_embedding_layer = nn.Embedding(n_items+1, fm_sizes[0])
        self._set_normalInit(self.item_embedding_layer, hasBias = False) 
        #self._set_xavierInit(self.item_embedding_layer, hasBias = False)
        #self._set_heInit(self.item_embedding_layer, hasBias = False) 
        for i in range(1, len(fm_sizes)):
            inChannel = 1 if i == 1 else n_fm
            #conv = nn.Conv2d(in_channels=inChannel, out_channels=32, kernel_size=fm_sizes[i]+myStride, stride=myStride)
            conv = nn.Conv2d(in_channels=inChannel, out_channels=n_fm, kernel_size=4, stride=myStride, padding=1)
            #self._set_normalInit(conv)
            #self._set_xavierInit(conv)
            self._set_heInit(conv)
            setattr(self, 'conv%i' % i, conv)
            self.convs.append(conv)

        self.predict = nn.Linear(n_fm, n_output)         # output layer
        self._set_xavierInit(self.predict)            # parameters initialization
        return
    
    def _set_xavierInit(self, layer, hasBias = True):
        init.xavier_uniform_(layer.weight)
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_heInit(self, layer, hasBias = True):
        init.kaiming_normal_(layer.weight, nonlinearity='relu')
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_normalInit(self, layer, parameter = [0.0, 0.1], hasBias = True):
        init.normal_(layer.weight, mean = parameter[0], std = parameter[1])
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_uniformInit(self, layer, parameter = 1, hasBias = True):
        init.uniform_(layer.weight, a = 0, b = parameter)
        if hasBias:
            init.uniform_(layer.bias, a = 0, b = parameter)
        return
    
    def forward(self, user, item_pos, item_neg, train = True):
        user = self.user_embedding_layer(user)
        item_pos = self.item_embedding_layer(item_pos)
        if train:
            item_neg = self.item_embedding_layer(item_neg)
        x1, x2 = None, None
        temp1, temp2 = list(), list() 
        out1, out2 = None, None
        for i in range(user.size()[0]):
            temp1.append(torch.mm(user[i].T, item_pos[i]))
            if train:
                temp2.append(torch.mm(user[i].T, item_neg[i]))
        x1 = torch.stack(temp1)
        x1 = x1.view(x1.size()[0], -1, x1.size()[1], x1.size()[2])
        if train:
            x2 = torch.stack(temp2)
            x2 = x2.view(x2.size()[0], -1, x2.size()[1], x2.size()[2])
        ''' ## conv2d -input  (batch_size, channel, weight, height) '''
        for conv in self.convs:
            x1 = torch.relu(conv(x1))
            if train:
                x2 = torch.relu(conv(x2))
        ''' ## conv2d -output (batch_size, out_channel, out_weight, out_height) '''
        x1 = torch.flatten(x1, start_dim = 1)
        x1 = self.dropout(x1)
        if train:
            x2 = torch.flatten(x2, start_dim = 1)
            x2 = self.dropout(x2)
        #out1 = torch.sigmoid(self.dropout(self.predict(x1)))
        out1 = torch.sigmoid(self.predict(x1))
        if train:
            #out2 = torch.sigmoid(self.dropout(self.predict(x2)))
            out2 = torch.sigmoid(self.predict(x2))
        return out1, out2

In [35]:
def my_loss(y1, y2):
    return torch.sum(torch.log(1+torch.exp(-(y1 - y2))))
class My_loss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, y1, y2):
        return torch.sum(torch.log(1+torch.exp(-(y1 - y2))))

In [36]:
model = ConvNCF(fm_sizes=FM_SIZE, n_fm=N_FM, n_users=N_USERS, n_items=N_ITEMS)
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay=0.001)
loss_func = My_loss()
if(torch.cuda.is_available()):
    model = model.cuda()
    loss_func = loss_func.cuda()
print(model)

ConvNCF(
  (dropout): Dropout(p=0.5, inplace=False)
  (user_embedding_layer): Embedding(6041, 64)
  (item_embedding_layer): Embedding(3953, 64)
  (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (predict): Linear(in_features=32, out_features=1, bias=True)
)


In [37]:
def getHitRatio(ranklist, gtItem):
    #HR击中率，如果topk中有正例ID即认为正确
    if gtItem in ranklist:
        return 1
    return 0

def getNDCG(ranklist, gtItem):
    #NDCG归一化折损累计增益
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return np.log(2) / np.log(i+2)
    return 0

def getH(ranklist1, ranklist2):
    L = len(ranklist1)
    common = len(list(set(ranklist1).intersection(set(ranklist2))))
    return 1-common/L

In [38]:
def movieEval_1(model, loss_func, utest, itest, topK = 100):
    if len(utest)==len(itest):
        n_users = len(utest)
    else:
        print('the length of test sets are not equal.')
        return
    hit = 0
    undcg = 0
    rank_all_users = list()
    #test_loss = list()
    hr = 0
    ndcg = 0
    with torch.no_grad():
        for i in range(n_users):
            map_item_score = dict()
            x1test = Variable(torch.from_numpy(utest[i].reshape(-1, 1)).type(torch.LongTensor))
            x2test = Variable(torch.from_numpy(itest[i].reshape(-1, 1)).type(torch.LongTensor))
            #y  = torch.from_numpy(ytest[i].reshape(-1, 1)).type(torch.FloatTensor)
            #x1test, x2test, y = x1test.cuda(), x2test.cuda(), y.cuda()
            x1test, x2test = x1test.cuda(), x2test.cuda()
            prediction, _ = model(x1test, x2test, None, train = False)
            #print(prediction)
            #loss = loss_func(prediction, y)
            #test_loss.append(loss.cpu().item())
            pred_vector = prediction.cpu().data.numpy().T[0]
            positive_item = itest[i][0]  # 取正例
            for j in range(len(itest[i])):
                map_item_score[itest[i][j]] = pred_vector[j]
            ranklist = heapq.nlargest(topK, map_item_score, key=map_item_score.get)
            rank_all_users.append(ranklist)
            hit += getHitRatio(ranklist, positive_item)
            undcg += getNDCG(ranklist, positive_item)
        #mean_test_loss = np.mean(test_loss)
        hr = hit / n_users
        ndcg = undcg / n_users
    #print('test_loss:', mean_test_loss)
    print('HR@', topK, ' = %.4f' % hr)
    print('NDCG@', topK, ' = %.4f' % ndcg)
    return hr, ndcg, rank_all_users

In [39]:
train_loss_list = list()
test_loss_list  = list()
hr_list = list()
ndcg_list = list()
for e in range(EPOCH):
    data = generate_train_batch(users_items_train, u_pos_test)
    train_loss = list()
    for train_batch in data:
        u = Variable(torch.from_numpy(train_batch[:,0].reshape(-1, 1)).type(torch.LongTensor))
        i = Variable(torch.from_numpy(train_batch[:,1].reshape(-1, 1)).type(torch.LongTensor))
        j = Variable(torch.from_numpy(train_batch[:,2].reshape(-1, 1)).type(torch.LongTensor))
        if (torch.cuda.is_available()):
            u, i, j = u.cuda(), i.cuda(), j.cuda()
        optimizer.zero_grad()
        yui, yuj = model(u, i, j)
        loss = loss_func(yui, yuj) 
        loss.backward()  
        train_loss.append(loss.cpu().item())
        optimizer.step()
    print('------第'+str(e+1)+'个epoch------')
    mean_train_loss = np.mean(train_loss)
    print('train_loss:', mean_train_loss)
    train_loss_list.append(mean_train_loss)
    if (e+1)%3 == 0:
        model.eval()
        hr, ndcg, rank_all_users = movieEval_1(model, loss_func, utest, itest)
        model.train()
        #test_loss_list.append(test_loss)
        hr_list.append(hr)
        ndcg_list.append(ndcg)

------第1个epoch------
train_loss: 348.95637766520184
------第2个epoch------
train_loss: 348.9585952758789
------第3个epoch------
train_loss: 348.8658803304036
HR@ 100  = 0.1386
NDCG@ 100  = 0.0294
------第4个epoch------
train_loss: 348.79567464192706
------第5个epoch------
train_loss: 347.4679590861003
------第6个epoch------
train_loss: 320.5345993041992
HR@ 100  = 0.4571
NDCG@ 100  = 0.1073
------第7个epoch------
train_loss: 289.3668390909831
------第8个epoch------
train_loss: 275.2731056213379
------第9个epoch------
train_loss: 266.8892822265625
HR@ 100  = 0.4927
NDCG@ 100  = 0.1344
------第10个epoch------
train_loss: 258.20362599690753
------第11个epoch------
train_loss: 255.66196950276694
------第12个epoch------
train_loss: 254.80242029825845
HR@ 100  = 0.4959
NDCG@ 100  = 0.1613
------第13个epoch------
train_loss: 257.0183970133464
------第14个epoch------
train_loss: 256.4720827738444
------第15个epoch------
train_loss: 256.8952166239421
HR@ 100  = 0.4909
NDCG@ 100  = 0.1737
------第16个epoch------
train_loss: 